<a href="https://colab.research.google.com/github/pietrodileo/Python_for_MD_thesis/blob/main/CNN_CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Import Libraries

In [79]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2

# !pip install tensorflow==2.8
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mimg
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)
from scipy import misc
import sklearn
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import wave
import pylab
import sys
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
from keras.utils.vis_utils import plot_model
import itertools
import time
import shutil
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
!pip install pyyaml h5py  # Required to save models in HDF5 format

!pip install -U kaleido
#need to restart runtime after install kaleido
import kaleido
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ⏰ Execution time

In [81]:
start_time = time.time()

# 🛫 Define Training Parameters

In [82]:
#@title Declare Constants
#@markdown ---
fileName = 'C1' #@param {type: "string"}
# K fold CrossValidation
k = 10 #@param {type: "number"}
num_cnn_layers = 4 #@param {type: "number"}
IMAGE_HEIGHT = 256 #@param {type: "number"}
IMAGE_WIDTH = 256 #@param {type: "number"}
BATCH_SIZE = 32 #@param {type: "slider", min: 1, max: 256}
EPOCHS = 100  #@param {type: "number"}
VAL_SPLIT = 0.2  #@param {type: "slider", min: 0, max: 1, step:0.05}
MODE = "grayscale"  #@param ['rgb', 'rgba', 'grayscale']
SaveFile = True #@param {type: "boolean"}
#@markdown ---

# 📑 Unzip Directory

In [83]:
TASK = str(EPOCHS) + 'EP_' + str(num_cnn_layers) + 'conv_' + 'Batch' + str(BATCH_SIZE) + '_adam_' + fileName  
TASK

'100EP_4conv_Batch32_adam_C1'

In [84]:
zipfile = fileName + '.zip'
directory = '/content/drive/MyDrive/TesiMagistrale/outputSpectrogram/CNN_TrainingIMGs/BW/'

zipPath = os.path.join(directory,zipfile)
outputDir = os.path.join(directory,'Risultati')
outputPath = os.path.join(outputDir,TASK)

if not os.path.exists(outputPath):
   # Create a new directory because it does not exist
   os.makedirs(outputPath)
   print("The output folder has been created!")

# Location of Zip File
drive_path = zipPath
local_path = '/content'

zipCopyPath = os.path.join(local_path,zipfile)
if not os.path.exists(zipCopyPath):
  # Copy the zip file and move it up one level (AKA out of the drive folder)
  !cp '{drive_path}' .
else:
  print('Files already transferred from Drive')

if not os.path.exists(fileName):
  # Navigate to the copied file and unzip it quietly
  os.chdir(local_path)
  !unzip -q '{zipfile}'
else:
  print('Files already unzipped')
# change directory to the new one
NewDir = os.path.join(local_path,fileName)

Files already transferred from Drive
Files already unzipped


# 🎨 Extract image path

In [85]:
valid_formats = [".png"]
pathlist = []
classnames = []
labVec = []
istanceVec = []
lablist = []
#dataclass = []
#dataAll = []
for folders in os.listdir(NewDir):
  # select a subfolder
  subfolder = os.path.join(NewDir,folders)
  # select all the records in the subfolder
  print('reading folder: '+ subfolder)
  subfolderList = []
  subfolderlabel = []
  pathVec = []
  for filename in os.listdir(subfolder):
    file_format = os.path.splitext(filename)[1] 
    if file_format.lower() in valid_formats:
      subfolderList.append(filename)
      currClass = os.path.split(subfolder)[1]
      labVec.append(currClass)
      istanceVec.append(filename)
      # extract file path
      pathVec.append(os.path.join(subfolder,filename))
      subfolderlabel.append(currClass)

  pathlist.append(pathVec)
  lablist.append(subfolderlabel)
  classnames.append(currClass)

reading folder: /content/C1/class_advanced_PD-OFF
reading folder: /content/C1/class_HealthyControl


##Information about Data

In [86]:
pathColumns = pd.DataFrame(pathlist)
pathColumns = pathColumns.transpose()
pathColumns.columns = classnames
pathColumns.head(5)

,class_advanced_PD-OFF,class_HealthyControl
0,/content/C1/class_advanced_PD-OFF/PD_OFF_TesiP...,/content/C1/class_HealthyControl/HC_TesiPDL_Vo...
1,/content/C1/class_advanced_PD-OFF/PD_OFF_TesiP...,/content/C1/class_HealthyControl/HC_TesiPDL_Vo...
2,/content/C1/class_advanced_PD-OFF/PD_OFF_TesiP...,/content/C1/class_HealthyControl/HC_TesiPDL_Vo...
3,/content/C1/class_advanced_PD-OFF/PD_OFF_TesiP...,/content/C1/class_HealthyControl/HC_TesiPDL_Vo...
4,/content/C1/class_advanced_PD-OFF/PD_OFF_TesiP...,/content/C1/class_HealthyControl/HC_TesiPDL_Vo...


In [87]:
labColumns = pd.DataFrame(lablist)
labColumns = labColumns.transpose()
labColumns.columns = classnames
labColumns.head(2)

,class_advanced_PD-OFF,class_HealthyControl
0,class_advanced_PD-OFF,class_HealthyControl
1,class_advanced_PD-OFF,class_HealthyControl


In [88]:
dictionary = {'Istance': istanceVec,'Class': labVec}
classVec = pd.DataFrame(data = dictionary)
classVec.head(4)

,Istance,Class
0,PD_OFF_TesiPDL_VowelE_0022_FineCut_Short_Pitch...,class_advanced_PD-OFF
1,PD_OFF_TesiPDL_VowelE_0040_FineCut_Short_Pitch...,class_advanced_PD-OFF
2,PD_OFF_TesiPDL_VowelE_0036_FineCut_Short_FreqM...,class_advanced_PD-OFF
3,PD_OFF_TesiPDL_VowelE_0048_FineCut_Short_Gauss...,class_advanced_PD-OFF


# 🔪 Split in K-Folds

Dividiamo ogni colonna di labColumns in K gruppi diversi. Divideremo ogni colonna con la stessa percentuale di split, così da essere sicuri di avere in ogni folder lo stesso numero di soggetti per ogni classe

In [89]:
from sklearn.model_selection import KFold
# perform crossvalidation 
testPath = []
y_trainClass = []
y_testClass = []
kfoldClass = []
splittedData = []
print('\nFound', len(classnames), 'classes, perform', k, '-fold Crossvalidation')
for i in range(0,len(classnames)):
  # define crossvalidation object
  kf = KFold(n_splits = k, shuffle = True, random_state = np.random.randint(1, 100))
  kfoldClass = []
  # take the current class vector
  currentClassVector = labColumns[classnames[i]]
  currentClassVector = currentClassVector.dropna()
  #print('Class: ', classnames[i])
  # split train and test set for each class
  for train_index , test_index in kf.split(currentClassVector):
    #print('train=', train_index, 'test=', test_index)
    trainPathClass = pathColumns[classnames[i]][train_index]
    testPathClass = pathColumns[classnames[i]][test_index]
    y_trainClass = labColumns[classnames[i]][train_index]
    y_testClass = labColumns[classnames[i]][test_index]
    # define dataframe for test and training for the current class
    dictionClassTrain = {'Path': trainPathClass,'Class': y_trainClass}
    dictionClassTest = {'Path': testPathClass,'Class': y_testClass}
    dfPathTest = pd.DataFrame(data = dictionClassTest)
    dfPathTrain = pd.DataFrame(data = dictionClassTrain)
    # make a tuple for the current folder (k) of the current class (i)
    kfoldTuple = (dfPathTest,dfPathTrain)
    # kfoldClass is a list containing k tuple regarding the i-th class
    kfoldClass.append(kfoldTuple)
  print('\nClass: ', classnames[i], '. Found ', len(currentClassVector), 'subjects')
  print('\n Training set contains', len(y_trainClass), 'subjects')
  print('\n Test set contains', len(y_testClass), 'subjects')
  # splittedData is a list containing a list for each class (the latter contains k tuple)
  splittedData.append(kfoldClass)


Found 2 classes, perform 10 -fold Crossvalidation

Class:  class_advanced_PD-OFF . Found  648 subjects

 Training set contains 584 subjects

 Test set contains 64 subjects

Class:  class_HealthyControl . Found  544 subjects

 Training set contains 490 subjects

 Test set contains 54 subjects


## Create Train Set and Test Set

In [90]:
TrainSet = []
TestSet = []
TrainPath = []
TestPath = []
TestPath1 = []
TrainPath1 = []
for i in range(0,k):
  for j in range(0,len(classnames)):
    # take the current class (list of k tuple)
    currClass = splittedData[j]
    # take the tuple regarding the k-th folder
    kfold = currClass[i]
    dfPathTest = kfold[0]
    dfPathTrain = kfold[1] 
    # for the first class
    if j == 0:
      TestPath1 = dfPathTest
      TrainPath1 = dfPathTrain
    # next classes
    elif j > 0:
      TestPath = pd.concat([TestPath1, dfPathTest],ignore_index=True)
      TrainPath = pd.concat([TrainPath1, dfPathTrain],ignore_index=True)
  # shuffle all the rows of train data 
  TrainPath = TrainPath.sample(frac=1)
  # save training set and test set for all the K-folders
  TrainSet.append(TrainPath)
  TestSet.append(TestPath)

## Get the train and test data

In [91]:
from tqdm import tqdm 

#A function to read and process the images to an acceptable format for our model
def read_and_process_image(list_of_images,nrows,ncolumns):
    """
    Returns X: 
        X is an array of resized images
    """    
    X = [] # images
    
    # Store all the images in the variable X
    for image in tqdm(list_of_images):
        X.append(cv2.resize(cv2.imread(image, cv2.COLOR_BGR2GRAY), 
                            (nrows,ncolumns), interpolation=cv2.INTER_CUBIC))  #Read the image
    
    return X

# Create new directories for test and training

In [92]:
def makeNewDirectory(fold,TrainSet,TestSet,local_path,fileName):
  # copy the image from the path to another directory
  ImageList = TrainSet[fold]['Path']
  ImageLabel = TrainSet[fold]['Class']
  ImageListTest = TestSet[fold]['Path']
  ImageLabelTest = TestSet[fold]['Class']
  foldNum = str(fold+1)
  newfolderpath = os.path.join(local_path,'newFolder_'+fileName+'_k-'+foldNum)
  if not os.path.exists(newfolderpath):
    # Create a new directory because it does not exist
    os.makedirs(newfolderpath)
    print("New folder has been created!")
  testsubfolder = os.path.join(newfolderpath,'Test')
  trainsubfolder = os.path.join(newfolderpath,'Training')
  if not os.path.exists(trainsubfolder):
    os.makedirs(trainsubfolder)
  if not os.path.exists(testsubfolder):
    os.makedirs(testsubfolder)
    print("Subfolders have been created!")
  return ImageList, ImageLabel, ImageListTest, ImageLabelTest, newfolderpath, testsubfolder, trainsubfolder

## Copy each image from the original folder to a new one (divided in subfolders representing each class)

In [93]:
import shutil
def CopyFile(dstn_subfolder, imageseries):
  for originalFilePath in imageseries:
    file_format = os.path.splitext(originalFilePath)[1] 
    filename = os.path.splitext(originalFilePath)[0] 
    fileclass = os.path.split(filename)[0]
    fileclass = os.path.split(fileclass)[1]
    filename = os.path.split(filename)[1]
    newfileFolder = os.path.join(dstn_subfolder,fileclass)
    if not os.path.exists(newfileFolder):
      # Create a new directory because it does not exist
      os.makedirs(newfileFolder)
      print("New folder has been created!")
    newFilePath = os.path.join(dstn_subfolder,fileclass,filename)
    newFilePath = newFilePath+file_format
    shutil.copy2(originalFilePath, newFilePath, follow_symlinks=True)

# 📈 Preparing the data
We can now load the spectrograms into memory. We use the image_dataset_from_directory utility to generate the datasets. The validation set is what will ultimately be our benchmark when becomes to performance and accuracy of our classifier. The seed is for reproducibility.

In [94]:
# Function to prepare our datasets for modelling
def prepare(batches):

  def normalize(img, label):
    return img / 255.0, label

  ds = (batches
        .map(normalize)
        .prefetch(tf.data.AUTOTUNE)
        ) 
  return ds


def ReadSetForKfold(trainsubfolder,testsubfolder,BATCH_SIZE,MODE,IMAGE_HEIGHT, IMAGE_WIDTH):
  # Create the dataset and select number of classes
  train_batches = tf.keras.preprocessing.image_dataset_from_directory(
                                            trainsubfolder,
                                            labels='inferred',
                                            class_names = False,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            color_mode= MODE,
                                            image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                            seed=0)
  # Create the dataset and select number of classes
  test_batches = tf.keras.preprocessing.image_dataset_from_directory(
                                            testsubfolder,
                                            labels='inferred',
                                            class_names = False,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            color_mode= MODE,
                                            image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                            seed=0)


  classNames = train_batches.class_names
  N_CLASSES = len(classNames)

  # Extract image path for training and validation set
  image_paths_train = train_batches.file_paths
  image_paths_valid = test_batches.file_paths

  train_dataset = prepare(train_batches)
  valid_dataset = prepare(test_batches)

  LabelNames = test_batches.class_names
  LabelNames
  return train_batches, test_batches, classNames, N_CLASSES, image_paths_train, image_paths_valid, train_dataset, valid_dataset, LabelNames

In [95]:
# Select number of channels
if MODE == 'grayscale':
  N_CHANNELS = 1
elif MODE == 'rgb':
  N_CHANNELS = 3
elif MODE == 'rgba':
  N_CHANNELS = 4

# 🧠 Make Model and Train

## 🦾 Define Model Architecture

In [96]:
def make_model(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS, N_CLASSES, num_cnn_layers):
  # Create CNN model with 3 Convolution Layer Architecture
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
  # Conv2D(NumFilter, FilterSize, option...)

  NUM_FILTERS = 32
  for i in range(1, num_cnn_layers+1):
    model.add(tf.keras.layers.Conv2D(NUM_FILTERS*i, (3,3), strides=(1, 1), activation='relu', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())

  # flattening
  model.add(tf.keras.layers.Flatten())
  # NN 
  model.add(tf.keras.layers.Dense(256, activation='relu')) 
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
  return model

## ♟ Set Optimizer and Loss Function 

In [97]:
monitorValue = 'val_loss'
monitorMode = 'min'

checkpoint_path = os.path.join(outputPath,'training_1/cp.ckpt')
bestModelName = os.path.join(outputPath,'best_mdl.h5')
earlyStopping = EarlyStopping(monitor=monitorValue, 
                              patience=40, 
                              verbose=0, 
                              mode=monitorMode,
                              restore_best_weights=True)

mcp_save = ModelCheckpoint(filepath=checkpoint_path,
                           save_best_only=True, 
                           monitor=monitorValue, 
                           mode=monitorMode)

reduce_lr_loss = ReduceLROnPlateau(monitor=monitorValue,
                                   factor=0.1, patience=5, 
                                   verbose=0, min_delta=1e-4, mode=monitorMode)

LOSS = 'sparse_categorical_crossentropy'

# OPTIMIZER = tf.keras.optimizers.SGD(
#         learning_rate=0.01,
#         momentum=0.0,
#         nesterov=False,
#         name='SGD')

OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## Confusion Matrix

In [98]:
def Evaluate_Batches(batch_dataset):
  #extract the labels of each batch
  true_labels = []
  pred_labels = [] 
  train_labels = []
  for image_batch, label_batch in batch_dataset:
    # image_batch contains n figure, where n is batch size.  
    # It's an EagerTensor with a shape of (n, ImHeight, ImWidth, Nchannels)
    preds = model.predict(image_batch,verbose=0)
    # preds is a ndarray with n rows and N_CLASSES columns
    # it is the output of the last neurons of the Dens/softmax Layer
    pred_labels += np.argmax(preds, axis = - 1).tolist()
    train_labels += label_batch.numpy().tolist()
    # Alternative Code (use EagerTensor, a bit longer) 
    # pred_labels.append(np.argmax(preds, axis = - 1))
    # pred_labels is a list of which each element is an ndarray with size n (batchsize)
    # This list is composed by M elements, where M is the number of batch 
  return train_labels, pred_labels

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=18)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45, fontsize=13)
        plt.yticks(tick_marks, target_names, fontsize=13)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     size='xx-large',
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]), 
                     size='xx-large',
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label',fontsize=14)
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass),fontsize=14)
    plt.show()    

def ConfMtxPlot(train_labels, pred_labels):
  confusion_mtx = confusion_matrix(train_labels, pred_labels) 
  # plot the confusion matrix
  f,ax = plt.subplots(figsize=(8, 7))
  sns.heatmap(confusion_mtx, annot=True, linewidths=0.1,cmap="Blues",linecolor="k", fmt= '.0f',ax=ax,cbar=False)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix")
  plt.show()

## 💪 Train the network 


In [ ]:
historyList = []
infoList = []
# Train the model with k-fold crossvalidation
for fold in range(0,k):
  print('\nCrossValidation: Epoch', fold+1)
  ImageList, ImageLabel, ImageListTest, ImageLabelTest, newfolderpath, \
   testsubfolder, trainsubfolder = \
   makeNewDirectory(fold,TrainSet,TestSet,local_path,fileName)
  CopyFile(trainsubfolder, ImageList)
  CopyFile(testsubfolder, ImageListTest)
  train_batches, test_batches, classNames, N_CLASSES, \
  image_paths_train, image_paths_valid, train_dataset, \
  valid_dataset, LabelNames = ReadSetForKfold(trainsubfolder,testsubfolder,\
                                              BATCH_SIZE,MODE,IMAGE_HEIGHT, IMAGE_WIDTH)
  # Lets plot the label to be sure we have the correct class
  sns.countplot(x = ImageLabel)
  plt.title('Labels')
  plt.show
  model = make_model(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS, N_CLASSES, num_cnn_layers)
  # Compile model
  model.compile(
    loss = LOSS,
    optimizer = OPTIMIZER,
    metrics=['accuracy'])
  # Train model and capture the history
  history = model.fit(train_dataset, 
                    epochs=EPOCHS, 
                    validation_data = valid_dataset,
                    callbacks = [earlyStopping, mcp_save, reduce_lr_loss]
                    )
  historyList.append(history)
  # Compute the final loss and accuracy
  final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
  print("\nFinal loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))
  runtime = time.time() - start_time
  print('\nRuntime:')
  print("--- %s seconds ---" % (runtime))
  print("--- %s minutes ---" % ((runtime)/60))
  # Save all the data to a dictionary
  # dictionary of lists  
  InfoDict = {'IMAGE_HEIGHT':[IMAGE_HEIGHT], 'IMAGE_WIDTH':[IMAGE_WIDTH],
              'BATCH_SIZE': [BATCH_SIZE], 'N_CHANNELS': [N_CHANNELS], 'N_CLASSES': [N_CLASSES],
              'EPOCHS':[EPOCHS], 'VAL_SPLIT': [VAL_SPLIT], 'MODE': MODE, 'K-Fold' : fold,
              'FINAL VALIDATION LOSS': [final_loss], 'FINAL VALIDATION ACC': [final_acc],
              'RUNTIME (s)': [runtime], 'RUNTIME (min)': [runtime/60]
              }
  infoList.append(InfoDict)
  pred_labels, true_labels = Evaluate_Batches(valid_dataset)
  confusion_mtx = confusion_matrix(true_labels, pred_labels)
  plot_confusion_matrix(confusion_mtx,
                          LabelNames,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False)
  # compute statistics from confusion matrix



CrossValidation: Epoch 1
Found 1181 files belonging to 2 classes.
Found 229 files belonging to 2 classes.
Epoch 1/100
37/37 [==============================] - ETA: 0s - loss: 0.9519 - accuracy: 0.6850

37/37 [==============================] - 34s 802ms/step - loss: 0.9519 - accuracy: 0.6850 - val_loss: 0.8631 - val_accuracy: 0.5415 - lr: 0.0010
Epoch 2/100
37/37 [==============================] - 20s 469ms/step - loss: 0.3131 - accuracy: 0.8747 - val_loss: 1.3626 - val_accuracy: 0.5415 - lr: 0.0010
Epoch 3/100
37/37 [==============================] - 19s 437ms/step - loss: 0.1669 - accuracy: 0.9356 - val_loss: 1.1130 - val_accuracy: 0.5415 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 17s 391ms/step - loss: 0.1321 - accuracy: 0.9577 - val_loss: 1.9085 - val_accuracy: 0.5415 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 18s 396ms/step - loss: 0.0871 - accuracy: 0.9678 - val_loss: 2.2848 - val_accuracy: 0.5415 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - ETA: 0s - loss: 0.0522 - accuracy: 0.9797

37/37 [==============================] - 24s 587ms/step - loss: 0.0522 - accuracy: 0.9797 - val_loss: 0.7763 - val_accuracy: 0.5939 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 17s 392ms/step - loss: 0.0560 - accuracy: 0.9780 - val_loss: 3.2902 - val_accuracy: 0.5415 - lr: 0.0010
Epoch 8/100
37/37 [==============================] - 30s 649ms/step - loss: 0.0434 - accuracy: 0.9797 - val_loss: 0.9215 - val_accuracy: 0.5721 - lr: 0.0010
Epoch 9/100
37/37 [==============================] - 18s 392ms/step - loss: 0.0318 - accuracy: 0.9881 - val_loss: 3.8168 - val_accuracy: 0.5546 - lr: 0.0010
Epoch 10/100
37/37 [==============================] - 19s 440ms/step - loss: 0.0413 - accuracy: 0.9848 - val_loss: 1.6401 - val_accuracy: 0.6463 - lr: 0.0010
Epoch 11/100
37/37 [==============================] - 19s 433ms/step - loss: 0.0278 - accuracy: 0.9941 - val_loss: 2.9722 - val_accuracy: 0.5633 - lr: 0.0010
Epoch 12/100
 7/37 [====>.........................] - ETA: 11s - los

In [ ]:
Summary = os.path.join(outputPath,'modelsummary.txt')
modelInfo = model.summary()

stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)
print(short_model_summary)

with open(Summary, 'w') as f:
  # Pass the file handle in as a lambda function to make it callable
  f.write(short_model_summary)

In [ ]:
pngOutput = os.path.join(outputPath,'model_plot.png')
plot_model(model, to_file=pngOutput, show_shapes=True, show_layer_names=False)

Plots to see how the models are performing

In [ ]:
colormap= plt.get_cmap('tab20')
colors = iter(colormap(np.linspace(0,1,10)))
epochs = range(1, len(historyList[0].history['accuracy'])+1)

for fold in range(0,k):
  color=next(colors)  
  plt.plot(epochs, historyList[fold].history['accuracy'], \
           label='Training Fold'+str(k),color=color)
  plt.plot(epochs, historyList[fold].history['val_accuracy'], \
           label='Val. Accuracy Fold'+str(k), linestyle = "dashdot",color=color)
plt.legend()
plt.grid()
plt.title("Train vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(range(1,EPOCHS))
plt.show()

In [ ]:
colormap= plt.get_cmap('tab20')
colors = iter(colormap(np.linspace(0,1,10)))

for fold in range(0,k):
  color=next(colors)  
  plt.plot(epochs, historyList[fold].history['loss'], \
           label='Training Fold'+str(k),color=color)
  plt.plot(epochs, historyList[fold].history['val_loss'], \
           label='Val. Accuracy Fold'+str(k), linestyle = "dashdot",color=color)
plt.legend()
plt.grid()
plt.title("Train vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(range(1,EPOCHS))
plt.show()